In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>
<br>
<br>
<br>

# **Feature Engineering: Redução da Quantidade de Variáveis**

Trabalhar com um grande volume de variáveis pode demandar um alto poder de processamento e disponibilidade de tempo maior. Sabendo que geralmente variáveis com alta quantidade de valores nulos não são úteis para o treinamento de um modelo de machine learning, o ideal é que a gente faça um corte nas variáveis com alto percentual de valores nulos, otimizando assim o tempo de processamento e a necessidade de um alto poder de processamento.
<br>
<br>
O critério adotado para a exclusão da variável será excluir as variáveis com **mais de 80% de valores nulos**. Vou excluir também todas as variáveis que tenham cardinalidade igual a 1.
<br>
<br>
Para essa etapa vamos usar o Python com a biblioteca Pandas.

<br>

## **Pacotes e Bibliotecas**

In [ ]:
# Importando bibliotecas.
import pandas as pd

<br>
<br>
<br>
<br>

## **Funções**

In [ ]:
# Função para verificar os metadados do conjunto de dados.
def generate_metadata_simples(df, ids, orderby = 'PC_NULOS'):
    '''
    Esta função retorna uma tabela com informações descritivas sobre um DataFrame.

    Parâmetros:
    - df: DataFrame que você quer descrever.
    - ids: Lista de colunas que são identificadores.
    - orderby: Ordenar por uma coluna específica.

    Retorna:
    Um DataFrame com informações sobre o df original.
    '''

    summary = pd.DataFrame({
        'USO_FEATURE': ['ID' if col in ids else 'Explicativa' for col in df.columns],
        'QT_NULOS': df.isnull().sum(),
        'PC_NULOS': round((df.isnull().sum() / len(df))* 100,2),
        'CARDINALIDADE': df.nunique(),
        'TIPO_FEATURE': df.dtypes
    })

    summary_sorted = summary.sort_values(by=orderby, ascending=False)
    summary_sorted = summary_sorted.reset_index()

    # Renomeando a coluna 'index' para 'FEATURES'.
    summary_sorted = summary_sorted.rename(columns={'index': 'FEATURE'})

    return summary_sorted



# =================================================================================================================================================== #



# Função para excluir variáveis com alto percentual de valores nulos e variáveis constantes.
def cut_variables(df, summary_sorted, cutoff_percentage=80, cutoff_cardinality=1):
    '''
    Esta função exclui as variáveis com alto percentual de valores nulos e variáveis constantes de um DataFrame.

    Parâmetros:
    - df: DataFrame que você quer descrever.
    - summary_sorted: Resultado dos metadados gerados.
    - cutoff_percentage: Limiar para o corte.
    - cutoff_cardinality: Limiar para a cardinalidade.

    Retorna:
    Um DataFrame com informações sobre o df original.
    '''

    # Criando conjuntos das variáveis que atendem aos critérios de valores nulos e baixa cardinalidade
    variables_to_drop_nulls = set(summary_sorted[summary_sorted['PC_NULOS'] > cutoff_percentage]['FEATURE'])
    variables_to_drop_cardinality = set(summary_sorted[summary_sorted['CARDINALIDADE'] <= cutoff_cardinality]['FEATURE'])

    # União dos conjuntos para evitar contagem duplicada
    variables_to_drop_union = variables_to_drop_nulls.union(variables_to_drop_cardinality)

    # Contando as variáveis que serão removidas
    num_variables_to_drop = len(variables_to_drop_union)

    # Cortando as variáveis do DataFrame original.
    df_cut = df.drop(variables_to_drop_union, axis=1)

    # Contando as variáveis que restaram no DataFrame.
    num_remaining_variables = len(df_cut.columns)

    # Formatando a saída.
    output = (f'Quantidade de variáveis com mais de {cutoff_percentage}% de valores nulos e com cardinalidade igual a {cutoff_cardinality}: {num_variables_to_drop}\n'
              f'Quantidade de variáveis restantes no Book de Variáveis: {num_remaining_variables}')

    return df_cut, output

<br>
<br>
<br>
<br>

## **Limpeza dos Books de Variáveis**

In [ ]:
# Gravando o diretório em uma variável.
dir = '/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars'

<br>

**book_01**

In [ ]:
# Lendo nossa tabela de 'book_previous_application_01'.
book_01 = pd.read_parquet(dir + '/book_previous_application_01', engine='pyarrow')

book_01.shape

(338857, 301)

In [ ]:
# Chamando a função de criação dos metadados.
book_01_metadata = generate_metadata_simples(book_01, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_01_metadata_filtrado = book_01_metadata[book_01_metadata['PC_NULOS'] > 80]
#book_01_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_01_clean, output = cut_variables(book_01, book_01_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 120
Quantidade de variáveis restantes no Book de Variáveis: 181


In [ ]:
# Salvando o novo book agora limpo.
book_01_clean.to_parquet(dir + '/book_clean/book_01')

<br>

**book_02**

In [ ]:
# Lendo nossa tabela de 'book_previous_application_02'.
book_02 = pd.read_parquet(dir + '/book_previous_application_02', engine='pyarrow')

book_02.shape

(338857, 1201)

In [ ]:
# Chamando a função de criação dos metadados.
book_02_metadata = generate_metadata_simples(book_02, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_02_metadata_filtrado = book_02_metadata[book_02_metadata['PC_NULOS'] > 80]
#book_02_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_02_clean, output = cut_variables(book_02, book_02_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 480
Quantidade de variáveis restantes no Book de Variáveis: 721


In [ ]:
# Salvando o novo book agora limpo.
book_02_clean.to_parquet(dir + '/book_clean/book_02')

<br>

**book_03**

In [ ]:
# Lendo nossa tabela de 'book_previous_application_03'.
book_03 = pd.read_parquet(dir + '/book_previous_application_03', engine='pyarrow')

book_03.shape

(338857, 1241)

In [ ]:
# Chamando a função de criação dos metadados.
book_03_metadata = generate_metadata_simples(book_03, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_03_metadata_filtrado = book_03_metadata[book_03_metadata['PC_NULOS'] > 80]
#book_03_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_03_clean, output = cut_variables(book_03, book_03_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 1350
Quantidade de variáveis restantes no Book de Variáveis: 1


In [ ]:
# Salvando o novo book agora limpo.
#book_03_clean.to_parquet(dir + '/book_clean/book_03')

Nesse caso não vou salvar o book pois só restou a variável de ID.

<br>

**book_04**

In [ ]:
# Lendo nossa tabela de 'book_previous_application_04'.
book_04 = pd.read_parquet(dir + '/book_previous_application_04', engine='pyarrow')

book_04.shape

(338857, 1241)

In [ ]:
# Chamando a função de criação dos metadados.
book_04_metadata = generate_metadata_simples(book_04, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_04_metadata_filtrado = book_04_metadata[book_04_metadata['PC_NULOS'] > 80]
#book_04_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_04_clean, output = cut_variables(book_04, book_04_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 1338
Quantidade de variáveis restantes no Book de Variáveis: 1


In [ ]:
# Salvando o novo book agora limpo.
#book_04_clean.to_parquet(dir + '/book_clean/book_04')

Nesse caso não vou salvar o book pois só restou a variável de ID.

<br>

**book_05**

In [ ]:
# Lendo nossa tabela de 'book_previous_application_05'.
book_05 = pd.read_parquet(dir + '/book_previous_application_05', engine='pyarrow')

book_05.shape

(338857, 1241)

In [ ]:
# Chamando a função de criação dos metadados.
book_05_metadata = generate_metadata_simples(book_05, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_05_metadata_filtrado = book_05_metadata[book_05_metadata['PC_NULOS'] > 80]
#book_05_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_05_clean, output = cut_variables(book_05, book_05_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 440
Quantidade de variáveis restantes no Book de Variáveis: 801


In [ ]:
# Salvando o novo book agora limpo.
book_05_clean.to_parquet(dir + '/book_clean/book_05')

<br>

**book_06**

In [ ]:
# Lendo nossa tabela de 'book_previous_application_06'.
book_06 = pd.read_parquet(dir + '/book_previous_application_06', engine='pyarrow')

book_06.shape

(338857, 1241)

In [ ]:
# Chamando a função de criação dos metadados.
book_06_metadata = generate_metadata_simples(book_06, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_06_metadata_filtrado = book_06_metadata[book_06_metadata['PC_NULOS'] > 80]
#book_06_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_06_clean, output = cut_variables(book_06, book_06_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 420
Quantidade de variáveis restantes no Book de Variáveis: 821


In [ ]:
# Salvando o novo book agora limpo.
book_06_clean.to_parquet(dir + '/book_clean/book_06')

<br>

**book_07**

In [ ]:
# Lendo nossa tabela de 'book_previous_application_07'.
book_07 = pd.read_parquet(dir + '/book_previous_application_07', engine='pyarrow')

book_07.shape

(338857, 1241)

In [ ]:
# Chamando a função de criação dos metadados.
book_07_metadata = generate_metadata_simples(book_07, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_07_metadata_filtrado = book_07_metadata[book_07_metadata['PC_NULOS'] > 80]
#book_07_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_07_clean, output = cut_variables(book_07, book_07_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 410
Quantidade de variáveis restantes no Book de Variáveis: 831


In [ ]:
# Salvando o novo book agora limpo.
book_07_clean.to_parquet(dir + '/book_clean/book_07')

<br>

**book_08**

In [ ]:
# Lendo nossa tabela de 'book_bureau_01'.
book_08 = pd.read_parquet(dir + '/book_bureau_01', engine='pyarrow')

book_08.shape

(305811, 1761)

In [ ]:
# Chamando a função de criação dos metadados.
book_08_metadata = generate_metadata_simples(book_08, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_08_metadata_filtrado = book_08_metadata[book_08_metadata['PC_NULOS'] > 80]
#book_08_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_08_clean, output = cut_variables(book_08, book_08_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 448
Quantidade de variáveis restantes no Book de Variáveis: 1313


In [ ]:
# Salvando o novo book agora limpo.
book_08_clean.to_parquet(dir + '/book_clean/book_08')

<br>

**book_09**

In [ ]:
# Lendo nossa tabela de 'book_bureau_02'.
book_09 = pd.read_parquet(dir + '/book_bureau_02', engine='pyarrow')

book_09.shape

(305811, 1921)

In [ ]:
# Chamando a função de criação dos metadados.
book_09_metadata = generate_metadata_simples(book_09, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_09_metadata_filtrado = book_09_metadata[book_09_metadata['PC_NULOS'] > 80]
#book_09_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_09_clean, output = cut_variables(book_09, book_09_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 576
Quantidade de variáveis restantes no Book de Variáveis: 1345


In [ ]:
# Salvando o novo book agora limpo.
book_09_clean.to_parquet(dir + '/book_clean/book_09')

<br>

**book_10**

In [ ]:
# Lendo nossa tabela de 'book_application_train'.
book_10 = pd.read_parquet(dir + '/book_application_train', engine='pyarrow')

book_10.shape

(215257, 12)

In [ ]:
# Chamando a função de criação dos metadados.
book_10_metadata = generate_metadata_simples(book_10, ids='SK_ID_CURR')

In [ ]:
# Mostrando os metadados apenas das variáveis com mais de 80% de valores nulos.
#book_10_metadata_filtrado = book_10_metadata[book_10_metadata['PC_NULOS'] > 80]
#book_10_metadata_filtrado

In [ ]:
# Chamadando a função de corte das variáveis.
book_10_clean, output = cut_variables(book_10, book_10_metadata, cutoff_percentage=80)
print(output)

Quantidade de variáveis com mais de 80% de valores nulos e com cardinalidade igual a 1: 0
Quantidade de variáveis restantes no Book de Variáveis: 12


In [ ]:
# Salvando o novo book agora limpo.
book_10_clean.to_parquet(dir + '/book_clean/book_10')

<br>

**Com essa limpeza saímos de 11391 variáveis criadas para 6017, uma redução de 52,82% de variáveis que não seriam úteis para o treinamento do nosso modelo, assim, a partir daqui, podemos otimizar o tempo de processamento das nossas tarefas.**